# Request Lifecycle in Phoenix

1. endpoint, initial plug path that all request go through, plug is piece of process over every request, endpoint itself is plug.
2. router, last plug in endpoint, that dispatching verb/path to controllers
3. controller, retrieve rquest information, talk with business domain, and prepare data for view layer, controller itself is plug too.
4. view, handle structured data from controller and conver to a presentation for user

# What to do when adding a Page

## Controller Action

```
def action(conn, params) do
  render(conn, "action.html", map)
end
```

## View(if required)

```
defmodule DemoWeb.HelloView do
  use DemoWeb, :view
end
```

connect controller action and template

## Template

```
<div class="phx-hero">
  <h2>Hello World, from <%= @messenger %>!</h2>
</div>
```


# Plug

The basic idea of plug is to unify the concept of a "connection" that we operate on.

Flavors:

1. function plugs
2. module plugs

## Function Plugs

```
def introspect(conn, _opts) do
  IO.puts """
  Verb: #{inpsect()conn.method}
  Host: #{inspect(conn.host}
  Headers: #{inspect(conn.req_headers)}
  """
  
  conn
end
```

1. accept a connection struct(`%Plug.Conn{}`) and options
2. return connection

## Module Plugs

```
defmodule HelloWeb.Plugs.Locale do
  import Plug.Conn

  @locales ["en", "fr", "de"]

  def init(default), do: default

  def call(%Plug.Conn{params: %{"locale" => loc}} = conn, _default) when loc in @locales do
    assign(conn, :locale, loc)
  end

  def call(conn, default) do
    assign(conn, :locale, default)
  end
end
```

1. `init/1` which initialize any arguments or options to be passed to `call/2`
2. `call/2` which carries out the conneciton transformation. `call/2` is just a function plug

## Where to plug

The endpoint, router and contoller in Phoenix accept plugs.

# Routing

## Resources

shortcut to generate restful route

## Path Helpers

```
HelloWeb.Router.Helpers.page_path(HelloWeb.Endpoint, :index)
```

## Scoped Routes

```
scope "/admin", HelloWeb.Admin as: :admin do
  pipe_through :browser
  
  resources "/reviews", ReviewController
end
```

## Pipelines

1. group group of plug
2. pipline itself is also a plug, so you can embed pipeline in other pipeline.

# Controller

1. Itermediary modules, function defined in it is action
2. Build on plug package

## Actions

Nameing conversion:

1. `index`
2. `show`
3. `new`, for rendering empty form
4. `create`, for create new item
5. `edit`, for rendering form with item data
6. `update`
7. `delete`

Parameters:
1. `conn`
2. `params`

## Rendering

1. `text`
2. `json`
3. `html`
4. `render`

## Layout

Layout is rendered by LayoutView in `layout_view.ex`

```
def index(conn, _params) do
  conn
  |> put_layout("admin.html")
  |> render("index.html")
end
```

## Overriding Rendering Formats

1. Following the template naming convention.
2. change action to this

```
  def index(conn, _params) do
    render(conn, :index)
  end
```

3. allow text format from router

```
plug :accepts, ["html", "text"]
```

## Redirection

```
redirect(conn, external: "https://elixir-lang.org/")
```

## Flash Messagae

```
    conn
    |> put_flash(:info, "hey welcome!")
    |> redirect(to: Routes.hello_path(conn, :index))

```

## Action Fallback

Define fallback controller

```
defmodule HelloWeb.MyFallbackController do
  use Phoenix.Controller

  def call(conn, {:error, :not_found}) do
    conn
    |> put_status(:not_found)
    |> put_view(HelloWeb.ErrorView)
    |> render(:"404")
  end

  def call(conn, {:error, :unauthorized}) do
    conn
    |> put_status(403)
    |> put_view(HelloWeb.ErrorView)
    |> render(:"403")
  end
end
```

Use fallback controller in normal controller

```
action_fallback HelloWeb.MyFallbackController
```

# Views and Templates

## Rendering Templates

Function defined in view can be used from template directly.

## Template Compilation

When a template is compiled into a view, it's simplely compiled as `render` function that expects two argumanets: the template name and the assgins

```
defmodule HelloWeb.PageView do
  use HelloWeb, :view

  def render("index.html", assigns) do
    "rendering with assigns #{inspect Map.keys(assigns)}"
  end
end
```
## Rendering Templates Mannually

```
Phoenix.View.render_to_string(DemoWeb.PageView, "test.html", message: "Hello from Iex!")
```

## Sharing views and templates

```
<%= render(HelloWeb.PageView, "test.html", message: "Hello from layout!") %>
<%= render("test.html", message: "Hello from sibling template!") %>
```


# Ecto

## Schema and Migration Generator

```
mix phx.gen.schema User users name:string email:string \
bio:string number_of_pets:integer
```

1. define schema
2. define db migration code

## Schema

Schema is Elixir structs

## Changesets and Validations

1. `cast/3`
2. `validate_required/3`

```
changeset = User.changeset(%User{}, %{})
changeset.valid?
changeset.errors
```

## Data Persistence

```
alias Hello.{Repo, User}
Repo.insert(%User{email: "user1@example.com"})
Repo.insert(%User{email: "user2@example.com"})
Repo.all(User)
```